<a href="https://colab.research.google.com/github/tuhanren/Logicgame-Torunament/blob/main/logicagme_tournament_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Section 1: Core Game Functionality**
This section focuses on implementing the core logic of the game, including:

1. **Number Generation and Validation**:
   - `generate_secret_number`: Responsible for generating a valid secret number.
   - `is_valid_guess`: Validates if the player's guess follows the game rules (e.g., no duplicate digits, within the valid range).
   - `evaluate_guess`: Compares the player's guess against the secret number and evaluates the result.

2. **Core Mechanics**:
   - These functions form the foundational logic for running the game, managing rules, and handling player input and scoring.

In [ ]:
import random
import time
from dataclasses import dataclass, field
from typing import List
from itertools import permutations


# Define player data class
@dataclass
class Player:
    name: str
    total_score: int = 0  # Total score
    total_attempts: int = 0  # Total attempts (only player attempts)
    total_time_taken: float = 0.0  # Total time taken
    round_score: int = 0  # Score for the current round
    round_attempts: int = 0  # Attempts for the current round (including player's attempts)
    round_time_taken: float = 0.0  # Time taken for the current round (in seconds)


@dataclass
class Tournament:
    players: List[Player] = field(default_factory=list)

    def register_player(self, name: str):
        """Register a new player"""
        player = Player(name)
        self.players.append(player)
        print(f"Player {name} has been registered.")

    def play_game_player_guesses(self, player: Player):
        """Game: Player guesses the computer's secret number"""
        secret_number = generate_secret_number()
        print(f"{player.name}, the game begins! The computer has generated a secret three-digit number: {secret_number}")
        attempts = 0
        start_time = time.time()
        while True:
            guess = input("Enter your guess (e.g., 123): ")

            if not is_valid_guess(guess):
                print("Invalid input! The number must be a three-digit number without zeros or duplicate digits. Please try again.")
                continue

            attempts += 1
            feedback = evaluate_guess(guess, secret_number)
            print(f"Feedback: {feedback}")
            if feedback == "+++":  # Guessed correctly
                end_time = time.time()
                time_taken = end_time - start_time
                player.round_time_taken = time_taken
                player.round_attempts += attempts
                player.total_attempts += attempts  # Update the player's total attempts
                score = self.calculate_score(attempts)
                player.round_score = score
                player.total_score += score
                player.total_time_taken += time_taken

                print(f"Congratulations! You guessed the secret number {secret_number} in {attempts} attempts!")
                print(f"You took {time_taken:.2f} seconds to complete the game.")
                break

    def play_game_computer_guesses(self, player: Player):
        """Optimized computer guessing algorithm"""
        groups = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
        answer_list = []
        attempts = 0  # Separate counter for computer's guesses

        while True:
            secret_number = input(f"{player.name}, please provide a three-digit secret number for the computer (no zeros or duplicate digits): ")
            if is_valid_guess(secret_number):
                break
            print("Invalid input! Please enter a three-digit number without zeros or duplicate digits.")

        # Phase 1: Group processing
        for group in groups:
            guess = ''.join(map(str, group))
            feedback = evaluate_guess(guess, secret_number)
            print(f"Computer's guess: {guess}, Feedback: {feedback}")
            attempts += 1

            if feedback == "":  # No match
                continue
            if feedback == "+++":  # Fully matched
                print(f"The computer guessed the secret number: {guess}")
                return attempts  # Return the number of computer's guesses

            for num in group:
                temp_guess = str(num) * 3
                temp_feedback = evaluate_guess(temp_guess, secret_number)
                print(f"Computer's guess: {temp_guess}, Feedback: {temp_feedback}")
                attempts += 1
                if temp_feedback != "":
                    answer_list.append(num)

                if len(answer_list) == 3:  # After finding three digits, adjust their positions
                    adjusted_list, attempts = self.adjust_positions(answer_list, secret_number, attempts)
                    final_guess = ''.join(map(str, adjusted_list))
                    print(f"The computer guessed the secret number: {final_guess}")
                    return attempts  # Return the number of computer's guesses

    def adjust_positions(self, answer_list: List[int], target: str, attempts: int) -> List[int]:
        """Adjust the positions in answer_list until feedback is +++"""
        for perm in permutations(answer_list):
            guess = ''.join(map(str, perm))
            feedback = evaluate_guess(guess, target)
            print(f"Computer tries permutation: {guess}, Feedback: {feedback}")
            attempts += 1
            if feedback == "+++":
                return list(perm), attempts
        return answer_list, attempts

    def calculate_score(self, attempts: int) -> int:
        """Calculate the score based on the number of attempts"""
        if attempts <= 3:
            return 3
        elif attempts <= 6:
            return 2
        elif attempts <= 9:
            return 1
        else:
            return 0

    def display_round_results(self):
        """Display results for the current round and the cumulative leaderboard"""
        # 1. Round leaderboard
        print("\nRound Leaderboard:")
        print(f"{'Rank':<5}{'Player':<8}{'Score':<8}{'Attempts':<10}{'Time (s)':<10}")
        round_leaderboard = sorted(
            self.players,
            key=lambda x: (-x.round_score, x.round_attempts, x.round_time_taken)
        )
        for rank, player in enumerate(round_leaderboard, 1):
            print(
                f"{rank:<6}{player.name:<10}{player.round_score:<10}{player.round_attempts:<10}{player.round_time_taken:<10.2f}"
            )

        # 2. Cumulative leaderboard
        print("\nCumulative Leaderboard:")
        print(f"{'Rank':<5}{'Player':<8}{'Score':<8}{'Attempts':<16}{'Time (s)':<15}")
        total_leaderboard = sorted(
            self.players,
            key=lambda x: (-x.total_score, x.total_attempts, x.total_time_taken)
        )
        for rank, player in enumerate(total_leaderboard, 1):
            print(
                f"{rank:<6}{player.name:<10}{player.total_score:<10}{player.total_attempts:<15}{player.total_time_taken:<15.2f}"
            )

    def play_tournament_round(self):
        """Each player plays one round, alternating between guessing and providing a number"""
        for player in self.players:
            print(f"\n{player.name}'s turn to guess the computer's number:")
            self.play_game_player_guesses(player)  # Player guesses the computer's number

            print(f"\nNow it's the computer's turn to guess {player.name}'s number:")
            computer_attempts = self.play_game_computer_guesses(player)  # Computer guesses the player's number
            print(f"The computer took {computer_attempts} attempts.")

            # Update the player's score, but do not include computer's guesses in player's data
            player.round_score += self.calculate_score(computer_attempts)
            player.total_score += self.calculate_score(computer_attempts)



def generate_secret_number() -> str:
    """Generate a three-digit number without duplicates or zeros"""
    digits = random.sample(range(1, 10), 3)
    return ''.join(map(str, digits))


def is_valid_guess(guess: str) -> bool:
    """Check if the guess is a valid three-digit number (no zeros or duplicates)"""
    if len(guess) != 3 or not guess.isdigit():
        return False
    if '0' in guess:
        return False
    if len(set(guess)) != len(guess):
        return False
    return True


def evaluate_guess(guess: str, secret_number: str) -> str:
    """Compare the player's guess to the secret number and provide feedback"""
    feedback = []
    secret_matched = [False] * 3
    guess_matched = [False] * 3

    for i in range(3):
        if guess[i] == secret_number[i]:
            feedback.append('+')
            secret_matched[i] = True
            guess_matched[i] = True

    for i in range(3):
        if not guess_matched[i]:
            for j in range(3):
                if not secret_matched[j] and guess[i] == secret_number[j]:
                    feedback.append('-')
                    secret_matched[j] = True
                    break

    return ''.join(feedback)


def main():
    print("Welcome to the Logic Game Tournament System!")
    tournament = Tournament()

    num_players = int(input("Enter the number of players: "))
    for i in range(num_players):
        name = input(f"Enter the name of Player {i + 1}: ")
        tournament.register_player(name)

    num_rounds = int(input("Enter the number of rounds: "))
    for _ in range(num_rounds):
        tournament.play_tournament_round()
        tournament.display_round_results()  # Display rankings and cumulative scores for this round


if __name__ == "__main__":
    main()


Welcome to the Logic Game Tournament System!
Enter the number of players: 1
Enter the name of Player 1: Andy
Player Andy has been registered.
Enter the number of rounds: 1

Andy's turn to guess the computer's number:
Andy, the game begins! The computer has generated a secret three-digit number: 382
Enter your guess (e.g., 123): 283
Feedback: +--
Enter your guess (e.g., 123): 382
Feedback: +++
Congratulations! You guessed the secret number 382 in 2 attempts!
You took 5.10 seconds to complete the game.

Now it's the computer's turn to guess Andy's number:
Andy, please provide a three-digit secret number for the computer (no zeros or duplicate digits): 259
Computer's guess: 123, Feedback: -
Computer's guess: 111, Feedback: 
Computer's guess: 222, Feedback: +
Computer's guess: 333, Feedback: 
Computer's guess: 456, Feedback: +
Computer's guess: 444, Feedback: 
Computer's guess: 555, Feedback: +
Computer's guess: 666, Feedback: 
Computer's guess: 789, Feedback: +
Computer's guess: 777, Feed

### **Section 2: Performance Analysis and Simulation**
This section focuses on analyzing and simulating the performance of the game logic:

1. **Performance Analysis**:
   - `find_worst_case`: Evaluates the worst-case performance by iterating through all possible valid secret numbers (likely 504 unique numbers, assuming three digits with no repeated digits).
   - Analyzes the efficiency and robustness of the guessing algorithm under edge cases.

2. **Testing and Benchmarking**:
   - Uses tools like `unittest.mock.patch` to simulate player inputs for controlled testing.
   - Incorporates timing functionality (via the `time` library) to measure performance.

In [ ]:
import time
from unittest.mock import patch
# from game_tournament_en import Tournament, Player, is_valid_guess

def find_worst_case():
    # Setup
    tournament = Tournament()
    player = Player(name="TestPlayer")
    tournament.register_player(player.name)

    worst_case = None
    max_attempts = 0
    max_time = 0

    # Iterate over all 504 possible valid secret numbers
    for secret_number in range(123, 988):  # From 123 to 987 inclusive
        secret_number_str = str(secret_number)

        # Ensure the number is valid (unique digits and within valid range)
        if not is_valid_guess(secret_number_str):
            continue

        # Mock input to provide the secret number
        with patch("builtins.input", return_value=secret_number_str):
            # Measure performance
            start_time = time.time()
            attempts = tournament.play_game_computer_guesses(player)
            end_time = time.time()

        time_taken = end_time - start_time

        # Update worst-case scenario
        if attempts > max_attempts or (attempts == max_attempts and time_taken > max_time):
            max_attempts = attempts
            max_time = time_taken
            worst_case = secret_number_str

        print(f"Tested Secret: {secret_number_str}, Attempts: {attempts}, Time: {time_taken:.5f}s")

    # Print the worst case
    print("\nWorst Case Found!")
    print(f"Secret Number: {worst_case}")
    print(f"Max Attempts: {max_attempts}")
    print(f"Max Time Taken: {max_time:.5f}s")

if __name__ == "__main__":
    find_worst_case()


Streaming output truncated to the last 5000 lines.
Computer tries permutation: 412, Feedback: +--
Computer tries permutation: 421, Feedback: +++
The computer guessed the secret number: 421
Tested Secret: 421, Attempts: 12, Time: 0.00101s
Computer's guess: 123, Feedback: ++
Computer's guess: 111, Feedback: 
Computer's guess: 222, Feedback: +
Computer's guess: 333, Feedback: +
Computer's guess: 456, Feedback: +
Computer's guess: 444, Feedback: +
Computer tries permutation: 234, Feedback: ---
Computer tries permutation: 243, Feedback: +--
Computer tries permutation: 324, Feedback: +--
Computer tries permutation: 342, Feedback: ---
Computer tries permutation: 423, Feedback: +++
The computer guessed the secret number: 423
Tested Secret: 423, Attempts: 11, Time: 0.00029s
Computer's guess: 123, Feedback: +
Computer's guess: 111, Feedback: 
Computer's guess: 222, Feedback: +
Computer's guess: 333, Feedback: 
Computer's guess: 456, Feedback: +-
Computer's guess: 444, Feedback: +
Computer's gues